In [1]:
!pip install spacy

In [2]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 11.4 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [3]:
!python -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.8.4) =================
ℹ spaCy installation:
/home/sambara127/Desktop/nyanyanya/venv/lib/python3.12/site-packages/spacy

NAME              SPACY            VERSION                            
ru_core_news_sm   >=3.8.0,<3.9.0   3.8.0   ✔



In [3]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install --no-cache-dir llama-cpp-python

In [4]:
from llama_cpp import Llama
from typing import Dict, List, Union
import spacy

In [5]:
llm = Llama(
    model_path="/home/sambara127/.lmstudio/models/mradermacher/Qwen-2.5-7B-Deep-Stock-v4-GGUF/Qwen-2.5-7B-Deep-Stock-v4.Q3_K_S.gguf",
    n_gpu_layers=-1,  # Полностью на GPU
    use_mlock=True,    # Фиксирует модель в RAM, если недостаточно VRAM
    use_mmap=True      # Ускоряет загрузку
)

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4050 Laptop GPU, compute capability 8.9, VMM: yes
llama_model_load_from_file_impl: using device CUDA0 (NVIDIA GeForce RTX 4050 Laptop GPU) - 5705 MiB free
llama_model_loader: loaded meta data with 50 key-value pairs and 339 tensors from /home/sambara127/.lmstudio/models/mradermacher/Qwen-2.5-7B-Deep-Stock-v4-GGUF/Qwen-2.5-7B-Deep-Stock-v4.Q3_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 7B Instruct
llama_model_loader: - kv   3:                            ge

In [6]:
nlp_pipeline = spacy.load("ru_core_news_sm")

In [7]:
print(Llama)

<class 'llama_cpp.llama.Llama'>


In [74]:
def generate_response(system_prompt, user_prompt):
    full_prompt = f"{system_prompt}\n{user_prompt}"

    # Задаём путь к модели
    model_path = "/home/sambara127/.lmstudio/models/mradermacher/Qwen-2.5-7B-Deep-Stock-v4-GGUF/Qwen-2.5-7B-Deep-Stock-v4.Q3_K_S.gguf"
    
    # Генерация ответа
    response = llm(full_prompt, max_tokens=20, temperature=0.8)
    # Выводим системный и пользовательский промт, а также ответ
    print("Системный промт:")
    print(system_prompt)
    
    print("\nПользовательский промт:")
    print(user_prompt)
    
    print("\nОтвет модели:")
    print(response['choices'][0]['text'])  # Берём текст из ответа
    
    return response


def normalize_text(s: str, nlp_pipeline: spacy.Language) -> str:
    """
    Нормализация текста с использованием NLP-пайплайна spaCy.

    Аргументы:
        s (str): Исходная строка текста.
        nlp_pipeline (spacy.Language): NLP-модель (пайплайн) spaCy для обработки текста.

    Возвращает:
        str: Нормализованный текст, где:
             - числа заменены на специальный токен <NUM>;
             - все слова приведены к лемме в нижнем регистре;
             - знаки пунктуации и стоп-слова удалены.
    """
    doc = nlp_pipeline(s)
    lemmas = [
        ('<NUM>' if token.like_num else token.lemma_.lower())
        for token in doc
        if not token.is_punct and not token.is_stop
    ]
    return ' '.join(lemmas) if lemmas else ''


def prepare_message_for_llm(text: Union[str, List[str]], categories: List[str]) -> str:
    """
    Формирует промты для LLM.

    Аргументы:
        text (Union[str, List[str]]): Один текст или список текстов для анализа.
        categories (List[str]): Список категорий, из которых нужно выбрать наиболее подходящую.

    Возвращает:
        str: Готовый промт в виде строки.
    """
    assert len(categories) >= 2, "Количество категорий должно быть 2 или больше."

    if isinstance(text, str):
        normalized_text = normalize_text(text, nlp_pipeline)
    else:
        normalized_text = "\n".join(normalize_text(t, nlp_pipeline) for t in text)

    # Формируем строку с категориями
    categories_as_string = ", ".join(categories[:-1]) + " и " + categories[-1]

    # system_prompt = (
    #     "Вы — модель для анализа отзывов. Ваша задача — анализировать текст и классифицировать его по заранее "
    #     "определённым категориям жалоб.\n"
    #     "Выдавайте ответ ТОЛЬКО в виде списка чисел (вектор вероятностей), без пояснений, комментариев или "
    #     "дополнительного текста.\n"
    #     "Формат ответа: [значение_1, значение_2, ..., значение_N]\n"
    #     "Где каждое значение — это число от 0 до 1, отражающее тональность и значимость категории.\n"
    #     "Сумма значений всех категорий должна быть равна 1.\n"
    #     f"Последняя категория ({categories[-1]}) - значение вероятности, если ни к одной из предыдущей категории не относится жалоба.\n"
    # )

    system_prompt = (
        "Вы — модель для анализа отзывов. Ваша задача — анализировать текст и классифицировать его по заранее определённым категориям.\n"
        "Выдавайте ответ ТОЛЬКО в виде списка чисел (вектор), без пояснений, комментариев или дополнительного текста.\n"
        "Формат ответа:\n"
        "[значение_1, значение_2, ..., значение_N]\n"
        "Где каждое значение — это число либо -1, либо 0, либо +1.\n"
        "- -1 = негативный акцент на категорию\n"
        "- 0 = нейтрально\n"
        "- +1 = положительный акцент на категорию\n"
        "Категории (по порядку в векторе):\n"
        f"{categories_as_string}\n"
        # f"Отдельно для {categories[:-1]}: (0 — есть жалоба, +1 — полный позитив, -1 — критика)\n"
        "Отвечайте строго в виде вектора без лишних слов.\n"
        # "Пример ответа: '[0.1, -0.5, 0.0, -0.3, -0.2, -1.0]'\n"
    )


    if isinstance(text, str):
        user_prompt = (
            f"Анализируй следующий отзыв и выдай только один вектор без дополнительного текста:\n"
            f"Категории: {categories_as_string}.\nТекст: {normalized_text.strip()}\nОтвет:"
        )
    else:
        user_prompt = "\n".join([
            f"Анализируй следующий отзыв и выдай только один вектор без дополнительного текста:\n"
            f"Категории: {categories_as_string}.\nТекст: {t.strip()}\nОтвет:"
            for t in normalized_text
        ])

    return system_prompt, user_prompt

In [78]:

# Пример использования
categories = ["пунктуальность ", "качество обслуживания ", "коммуникация с клиентами", "комфорт во время полёта", "cтоимость билетов"]
text = "Обслуживание было отличным. Еды было достаточно, и она была вкусной (2 полноценных приема пищи, а также закуски и напитки, когда захотите). Взлет и посадка прошли отлично. Бортпроводники в целом были приятными. Были некоторые недостатки в полете, такие как очень медленная развлекательная система, большая задержка более чем на 3 часа. В целом полет прошел хорошо, за исключением задержки в аэропорту Владивостока."

# balast_category = 'полная оценка'
# categories.append(balast_category)

system_prompt, user_prompt = prepare_message_for_llm(text, categories)

response = generate_response(system_prompt, user_prompt)


Llama.generate: 367 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     335.50 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =     531.96 ms /    20 runs   (   26.60 ms per token,    37.60 tokens per second)
llama_perf_context_print:       total time =     541.04 ms /    21 tokens


Системный промт:
Вы — модель для анализа отзывов. Ваша задача — анализировать текст и классифицировать его по заранее определённым категориям.
Выдавайте ответ ТОЛЬКО в виде списка чисел (вектор), без пояснений, комментариев или дополнительного текста.
Формат ответа:
[значение_1, значение_2, ..., значение_N]
Где каждое значение — это число либо -1, либо 0, либо +1.
- -1 = негативный акцент на категорию
- 0 = нейтрально
- +1 = положительный акцент на категорию
Категории (по порядку в векторе):
пунктуальность , качество обслуживания , коммуникация с клиентами, комфорт во время полёта и cтоимость билетов
Отвечайте строго в виде вектора без лишних слов.


Пользовательский промт:
Анализируй следующий отзыв и выдай только один вектор без дополнительного текста:
Категории: пунктуальность , качество обслуживания , коммуникация с клиентами, комфорт во время полёта и cтоимость билетов.
Текст: обслуживание отличный еды достаточный вкусный <NUM> полноценный приём пища закуска напиток захотите взлёт

In [14]:
user_prompt

'Анализируй следующий отзыв и выдай только один вектор вероятностей без дополнительного текста:\nКатегории: пунктуальность , качество обслуживания , коммуникация с клиентами, комфорт во время полёта, cтоимость билетов и полная оценка.\nТекст: рейс постоянно задерживаться устать тратить время аэропорт цена космос\nОтвет:'

In [11]:
system_prompt

'Вы — модель для анализа отзывов. Ваша задача — анализировать текст и классифицировать его по заранее определённым категориям жалоб.\nВыдавайте ответ ТОЛЬКО в виде списка чисел (вектор вероятностей), без пояснений, комментариев или дополнительного текста.\nФормат ответа: [значение_1, значение_2, ..., значение_N]\nГде каждое значение — это число от 0 до 1, отражающее тональность и значимость категории.\nСумма значений всех категорий должна быть равна 1.\nПоследняя категория (полная оценка) - значение вероятности, если ни к одной из предыдущей категории не относится жалоба.\n'

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install --no-cache-dir llama-cpp-python --force-reinstall
